In [4]:
import os
os.environ["PATH"] += os.pathsep + r"/Users/mmymacymac/Developer/Tools/ffmpeg-2025-08-18-git-0226b6fb2c-full_build/bin"

In [5]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True,
    chunk_length_s=10,
    stride_length_s=2,
)

sample = "../audio/calling02.mp3"

result = pipe(sample)

print(result)

Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


{'text': ' 한국국토정보공사 한국국토정보공사 한국국토정보공사 한국국토정보공사 한국국토정보공사 한국국토정보공사 한국국토정보공사 한국국토정보공사 한국국토정보공사 한국국토정보공사 상당히 좋다. 영영, 선생님에게 연결 중입니다. 잠시만 기다려주십시오. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. 상담사와 연결되었습니다 여보세요? 여보세요? 안녕하세요? 그래요? 이름이 뭐예요? 어 송유진이에요 어 유진아 몇 년생이고? 저 01년 11월 6일생에 14시 18분 출신 유진이 유진이. 유진아 뭐가 제일 궁금하노? 저 올해 전체적인 운세가 조금 궁금하고요 그리고 제가 이제 지금 일을 그만두고 이제 반년 정도 취업을 위해가지고 아예 새로운 거� 걸 공부할 건데 이게 잘 저한테 맞을지 잘 모르겠더라고요. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고? Q. 새로운 공부는 어떤 일을 하려고

In [6]:
start_end_text = []
for chunk in result["chunks"]:
    start = chunk["timestamp"][0]
    end = chunk["timestamp"][1]
    text = chunk["text"]
    start_end_text.append([start, end, text])

import pandas as pd
df = pd.DataFrame(start_end_text, columns=["start", "end", "text"])
df.to_csv("calling02.csv", index=False, sep="|")
display(df)

,start,end,text
0,0.0,1.0,한국국토정보공사
1,1.0,2.0,한국국토정보공사
2,2.0,3.0,한국국토정보공사
3,3.0,4.0,한국국토정보공사
4,4.0,5.0,한국국토정보공사
...,...,...,...
225,508.0,511.0,아마 안 움직이면 그게 계속 있을 것 같은데
226,511.0,512.0,아 그래요?
227,512.0,514.0,네 알겠습니다
228,514.0,516.0,어 그래 화이팅해라
